### SGD

In [1]:
# Stochastic Gradient Descent

class SGD:
    
    def __init__(self, lr = 0.01):
        
        self.lr = lr
        
        
    def update(self, params, grads):
        
        for key in params.keys():
            
            params[key] -= self.lr * grads[key]

### SGD의 단점: 비등방성(anisotropy) 함수에서 탐색 경로가 비효율적
- ex. '홀로그램' 과 같이 특정한 방향으로 보면 평소와 다른 성질이 보이는 경우

#### SGD의 대안, Momentum, AdaGrad, Adam
#### Ref: https://light-tree.tistory.com/140  // https://tensorflow.blog/2017/03/22/momentum-nesterov-momentum/ // https://jjeongil.tistory.com/999


### Momentum

In [2]:
# 기하학 관점에서 해석하면, 
# 이전 업데이트 단계의 W 에 대한 손실함수의 기울기 값( 정확하게는 기울기 값 * 학습률에 momentum( 보통 0.9 ) 을 곱한 값 ) ) )과 현 단계에서의 값을 합하여 좀 더 큰 흐름의 방향으로 바꿔 움직임을 크게 만든다.

class Momentum:
    
    def __init__(self, lr = 0.01, momentum = 0.9):
        
        self.v = None
        self.lr = lr
        self.momentum = momentum
        
    
    def update(self, params, grads):
        
        if self.v == None:
            
            self.v = {}
            
            for key, val  in params.items():
                self.v[ key ] = np.zeros_like( val )
                
        for key in params.keys():
            self.v[ key ] = self.momentum * self.v[ key ] - self.lr * grads[ key ]
            params[ key ] += self.v[key]
                

### AdaGrad

In [3]:
# 학습률에 따른 학습 시간 혹은 발산 문제 발생
# 각각의 매개변수( 가중치값 )에 맞게 적응적으로 학습률을 조정하여 위의 문제를 해결함.
# 매개변수의 원소 중 크게 갱신된 원소의 학습률을 낮춤.

class AdaGrad:
    
    def __init__(self, lr = 0.01):
        
        self.lr = 0.01
        self.h = None
    
    
    def update(self, params, grads):
        
        if self.h == None:
            
            self.h = {}
            
            for key, val in params.items():
                
                self.h[ key ] = np.zeros_like(val)
            
        for key in params.keys():
            
            self.h[ key ] += grads[ key ] * grads[ key ]
            params[ key ] -= self.lr * grads[ key ] / ( np.sqrt( self.h[ key ] ) + 1e-7 )

### RMSProp 

In [4]:
# AdaGrad로 학습이 무힌하게 이루어진다면, 어느 순간 갱신량이 0이 되는 문제가 발생
# RMSProp은 '지수이동평균'을 이용하여 과거 기울기의 반영 규모를 기하급수적으로 감소시킴.

class RMSProp:
    
    def __init__(self, lr = 0.01, decay_rate = 0.9 ):
        
        self.lr = 0.01
        self.decay_rate = decay_rate
        self.h = None
    
    
    def update(self, params, grads):
        
        if self.h is None:
            
            self.h = {}
            
            for key, val in params.items():
                
                self.h[ key ] = np.zeros_like(val)
            
        for key in params.keys():
            
            self.h[ key ] *= self.decay_rate
            self.h[ key ] += (1 - self.decay_rate) * grads[ key ] * grads[ key ]
            params[ key ] -= self.lr * grads[ key ] / ( np.sqrt( self.h[ key ] ) + 1e-7 )

### Adam
- ref: https://hiddenbeginner.github.io/deeplearning/2019/09/22/optimization_algorithms_in_deep_learning.html

In [5]:
# Momentum( 이전 업데이트를 고려 ) 과 AdaGrad( 적응적으로 학습률 조정 )를 융합한 기법

class Adam:
    
    def __init__(self, lr = 0.001, beta1 = 0.9, beta2 = 0.999):
        
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.m = None
        self.v = None
        self.iter = 0
    
    
    def update(self, params, grads):
        
        if self.m is None:
            
            self.m, self.v = {}, {}
            
            for key, val in params.items():
                
                self.m[ key ] = np.zeros_like( val )
                self.v[ key ] = np.zeros_like( val )
        
        self.iter += 1
        lr_t = self.lr * np.sqrt( 1.0 - self.beta2 ** self.iter ) / (1.0 - self.beta1 ** self.iter) 
            
        for key in params.keys():
            
            self.m[ key ] += (1 - self.beta1) * (grads[ key ] - self.m[ key ])
            self.v[ key ] += (1 - self.beta2) * (grads[ key ]**2 - self.v[ key ])
            
            params[ key ] -= lr_t * self.m[ key ] / (np.sqrt( self.v[ key ] ) + 1e-7 )        